In [14]:
# imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
# google trends
# Keyword: "Auto Finanzierung"
gt_auto_finanzierung_1 = pd.read_csv('./original_data/gt_auto_finanzierung_1.csv')
gt_auto_finanzierung_2 = pd.read_csv('./original_data/gt_auto_finanzierung_2.csv')
gt_auto_finanzierung_3 = pd.read_csv('./original_data/gt_auto_finanzierung_3.csv')
# Keyword: "Auto Leasing"
gt_auto_leasing_1 = pd.read_csv('./original_data/gt_auto_leasing_1.csv')
gt_auto_leasing_2 = pd.read_csv('./original_data/gt_auto_leasing_2.csv')
gt_auto_leasing_3 = pd.read_csv('./original_data/gt_auto_leasing_3.csv')
# Keyword: "BMW Finanzierung"
gt_bmw_finanzierung_1 = pd.read_csv('./original_data/gt_bmw_finanzierung_1.csv')
gt_bmw_finanzierung_2 = pd.read_csv('./original_data/gt_bmw_finanzierung_2.csv')
gt_bmw_finanzierung_3 = pd.read_csv('./original_data/gt_bmw_finanzierung_3.csv')
# Keyword: "BMW Leasing"
gt_bmw_leasing_1 = pd.read_csv('./original_data/gt_bmw_leasing_1.csv')
gt_bmw_leasing_2 = pd.read_csv('./original_data/gt_bmw_leasing_2.csv')
gt_bmw_leasing_3 = pd.read_csv('./original_data/gt_bmw_leasing_3.csv')
# Keyword: "Leasing"
gt_leasing_1 = pd.read_csv('./original_data/gt_leasing_1.csv')
gt_leasing_2 = pd.read_csv('./original_data/gt_leasing_2.csv')
gt_leasing_3 = pd.read_csv('./original_data/gt_leasing_3.csv')
# Keyword: "BMW"
gt_bmw_1 = pd.read_csv('./original_data/gt_bmw_1.csv')
gt_bmw_2 = pd.read_csv('./original_data/gt_bmw_2.csv')
gt_bmw_3 = pd.read_csv('./original_data/gt_bmw_3.csv')

In [ ]:
# apgi

In [ ]:
# arbeitslosenzahlen

In [ ]:
# bruttoinlandsprodukt

In [ ]:
# euribor

In [ ]:
# gebrauchtwagenpreiseindex

In [ ]:
# ifo beschäftigungsbarometer

In [ ]:
# ifo geschäftsklima

In [ ]:
# bmw absatzzahlen